In [45]:
from nltk import ngrams
import nltk as nltk
from nltk.corpus import stopwords

#nltk.download('punkt');
#nltk.download('averaged_perceptron_tagger');
from nltk.stem import WordNetLemmatizer;
wordnet_lemmatizer = WordNetLemmatizer();
#nltk.download('wordnet');

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np

import re
import os
import string
import operator
from itertools import islice
from collections import Counter

from datetime import date
import matplotlib.dates as dates

In [46]:
sns.set_style('darkgrid')
sns.set_context('paper')
pd.set_option('display.max_colwidth', None)

In [47]:
punct_signs = list(string.punctuation)
punct_signs.append('…')
punct_signs.append('¿')
punct_signs.append('•')
punct_signs.append('”')
punct_signs.append('“')
punct_signs.append('–')
punct_signs.remove('&')
punct_signs.append('∑')

stop_words = stopwords.words('english') + ['also', 'could', ]

In [48]:
df = pd.read_csv('data/courses.csv')
df = df.fillna("")
df.columns

Index(['ID', 'Country', 'Univeristy', 'G2RWorldRank', 'G2RNationalRank',
       'QSWorldUniversityRanking2021', 'Program',
       'SchoolOfferingCourseProgram', 'CourseDescription', 'Course Unit',
       'CourseUnitGoals', 'CourseUnitOutcomes'],
      dtype='object')

In [49]:
import math
def clean_text(text):
    for p in punct_signs:
        text = text.replace(p, ' ')
    clean_text = text.lower().split()
    clean_text = [w for w in clean_text if w not in stop_words]
    return clean_text

def get_pairs(lst):
    i = 0
    res = []
    while i < len(lst):
        j = i + 1
        while j < len(lst):
            res.append((lst[i], lst[j]))
            j = j + 1
        i = i +1
    return res

In [50]:
data = '. '.join(list(df['CourseUnitOutcomes']))

In [51]:
goals = data.split('|')
ids = 0
pairs = []
for goal in goals:
    tokens = clean_text(goal)
    pos_tags = nltk.pos_tag(tokens)
    filtered_tags = [wordnet_lemmatizer.lemmatize(pt[0], pos="v") 
                     for pt in pos_tags if pt[1].startswith('V') or 
                    pt[1].startswith('N') or pt[1].startswith('J')]
    pairs += get_pairs(filtered_tags)

In [52]:
edges = pd.DataFrame(pairs, columns =['Source', 'Target'])

In [53]:
edges.shape

(31373, 2)

In [54]:
df = edges.reset_index().groupby(['Source', 'Target']).agg({'index' : 'count'})

In [59]:
df = df.reset_index()

In [60]:
df.columns = ['Source', 'Target', 'Weigth']
df.to_csv('data/net_coocurrence.csv')

In [61]:
df.head()

,Source,Target,Weigth
0,abilities,critical,1
1,abilities,current,1
2,abilities,evaluation,1
3,abilities,methodologies,1
4,abilities,research,2
